# Aula 1 - Buscando a similaridade

In [1]:
url = 'https://raw.githubusercontent.com/Mirlaa/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [2]:
import pandas as pd

dados = pd.read_csv(url)

## Similaridade entre as palavras

In [3]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language = 'portuguese')

In [6]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 62.4 MB/s eta 0:00:00


In [8]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences = dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

## Similaridade entre sentenças

In [9]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [10]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]

In [11]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)

In [12]:
vetor_inferido = modelo.infer_vector(['entrega'])

In [13]:
frases_similares = modelo.dv.most_similar([vetor_inferido],topn=5)

# Aula 2 - Ajustando os textos de review

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
import re
from nltk.corpus import stopwords

def tratamento_inicial (texto):
  texto = re.sub(r'\W',' ',texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])

In [16]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

## Uniformizando o texto

In [17]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.9 MB/s eta 0:00:00


In [18]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [19]:
dados['tratamento_2'] = sem_acentos

In [20]:
dados['review_token'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)

## Correções nos dados

In [21]:
df = dados.drop_duplicates(subset= ['tratamento_2'])

In [22]:
df = df[df['tratamento_2'] != '']

In [23]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

In [24]:
df.reset_index(drop=True, inplace=True)

## Lematização

In [25]:
!pip install stanza
!python -m stanza.download pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 37.5 MB/s eta 0:00:00
/usr/bin/python3: No module named stanza.download


In [26]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt',
                      processors='tokenize,lemma',
                      use_gpu=False,
                      batch_size= 64,
                      n_process = 4)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: pt (Portuguese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package         |
-------------------------------
| tokenize  | bosque          |
| mwt       | bosque          |
| lemma     | bosque_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [27]:
def lematizar_texto(textos):
  texto_lematizados = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizados.append(frase_lematizada)

  return texto_lematizados

In [28]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

['gostar muito experiencia comprar',
 'meu filha gostar produto',
 'compra ser facil compra rapida']

In [29]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])

# Aula 3 - Otimizando o Doc2vec

In [30]:
df_lem = df.drop_duplicates(subset=['tratamento_4'])

In [31]:
df_lem.reset_index(drop=True, inplace=True)

In [32]:
df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in df_lem.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)

/tmp/ipython-input-2601368758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)


## Treinamento do Doc2vec

In [33]:
modelo = Doc2Vec(vector_size = 300, min_count = 2,
                 window=5, workers = 1, seed=45, epochs = 15)
modelo.build_vocab(dados_tag)

modelo.train(dados_tag, total_examples=modelo.corpus_count,
             epochs= modelo.epochs)

## Verificando a similardade - Doc2Vec

# Aula 4 - Considerando o significado das frases

In [34]:
modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

In [35]:
import tensorflow_hub as hub

modelo = hub.load(modulo_url)

In [36]:
reviews = df['tratamento_3'].tolist()
reviews_emb = modelo(reviews)

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(reviews_emb)

array([[1.0000002 , 0.61586297, 0.5175547 , ..., 0.5427756 , 0.57541025,
        0.6691027 ],
       [0.61586297, 0.99999994, 0.5666107 , ..., 0.63920605, 0.6011834 ,
        0.6711744 ],
       [0.5175547 , 0.5666107 , 0.9999999 , ..., 0.58997375, 0.6564975 ,
        0.5922985 ],
       ...,
       [0.5427756 , 0.63920605, 0.58997375, ..., 1.        , 0.6378722 ,
        0.61628115],
       [0.57541025, 0.6011834 , 0.6564975 , ..., 0.6378722 , 1.0000001 ,
        0.6389235 ],
       [0.6691027 , 0.6711744 , 0.5922985 , ..., 0.61628115, 0.6389235 ,
        1.        ]], dtype=float32)

## Avaliando as sentenças similares

In [38]:
import numpy as np
def sentencas_similares(tema, reviews, reviews_emb, top_n=5):
  tema_emb = modelo([tema])
  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()

  indices_similares = np.argsort(-similaridades)

# Aula 5 - Transformando a similaridade em aplicação

In [ ]:
#!pip install gradio

In [40]:
import gradio as gr
nltk.download('punkt')
nltk.download('stopwords')

modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
modelo = hub.load(modulo_url)

def tratar_texto(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  word_tokenize_pt = partial(word_tokenize, language='portuguese')
  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')
  tokens = [w for w in tokens if w not in stop_words]

  texto_sem_acentos = unidecode.unidecode(' '.join(tokens))

  texto_normalizado = re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto_sem_acentos)
  return texto_normalizado

def process_csv(arquivo, tema):
  df = pd.read_csv(arquivo.name)

  df['review_tratada'] = df['review'].apply(tratar_texto)
  df = df[df['review_tratada']!='']
  df.drop_duplicates(subset=['review_tratada'],inplace=True)

  reviews_emb = modelo(df['review_tratada'].tolist())
  tema_emb = modelo([tema])

  similaridades = cosine_similarity(tema_emb,reviews_emb).flatten()
  top_indices = np.argsort(-similaridades)

  similar_reviews = df[['nota_review','review']].iloc[top_indices]

  similar_df = similar_reviews.head(200)

  nome_arquivo = f'reviews_similares_{tema}.csv'
  similar_reviews.to_csv(nome_arquivo)
  return similar_df, nome_arquivo

with gr.Blocks() as app:
  with gr.Row():
    gr.Markdown('## Encontrando as reviews mais similares ao tema')
  csv_entrada = gr.File(label='Envie o CSV com as reviews', file_types=['.csv'])
  tema_entrada = gr.Textbox(label='Digite o tema para busca (ex: "entrega")')

  botao = gr.Button('Clique para buscar as reviews')

  tabela_saida = gr.Dataframe(label='Top 200 reviews similares', headers=['Nota', 'Reviews'], interactive=False)
  arquivo_saida = gr.File(label='Baixar CSV ordenado com as reviews mais similares ao tema', interactive =False)

  botao.click(process_csv, inputs=[csv_entrada,tema_entrada], outputs=[tabela_saida,arquivo_saida])
app.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://988574912d3c17e9e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
